
# Diabetes Dataset Analysis

This notebook analyzes the diabetes dataset using various data visualization techniques and machine learning models.


In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc

# Load the dataset
df = pd.read_csv('/mnt/data/diabetes.csv')

# Explore the data
df.describe()



### Data Exploration

The dataset contains information about diabetes patients. Let's explore the data to understand the distribution of features.


In [ ]:

# Plot the distribution of the target variable (Outcome)
plt.figure(figsize=(6, 4))
sns.countplot(df['Outcome'], palette='coolwarm'])
plt.title('Distribution of Outcome Variable')
plt.show()



The above plot shows the distribution of the `Outcome` variable, which indicates whether a patient has diabetes (`1`) or not (`0`).


In [ ]:

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()



The correlation matrix above shows the relationship between different features in the dataset. Higher absolute values indicate stronger correlations.



### Data Splitting

Next, we will split the dataset into training and testing sets to evaluate the performance of our machine learning model.


In [ ]:

# Split the data into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### Model Training

We will use a Random Forest classifier and apply GridSearchCV to find the best hyperparameters for our model.


In [ ]:

# Train the Random Forest model with GridSearchCV for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters from GridSearch
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')



### Model Evaluation

After training the model, we will evaluate its performance on the test data using accuracy, confusion matrix, and classification report.


In [ ]:

# Make predictions on the test set
y_pred = grid_search.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')



### ROC-AUC Curve

The ROC-AUC curve is a graphical representation of the model's performance, illustrating the trade-off between true positive rate (sensitivity) and false positive rate (1-specificity).


In [ ]:

# Plot the ROC-AUC curve
y_pred_prob = grid_search.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
